<a href="https://colab.research.google.com/github/ArjunBalaji79/dockingscore_app_streamlit/blob/main/GCN_CCG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import all neccesary Libraries

In [ ]:
! pip install torch_geometric
! pip install scipy
! pip install rdkit
import sklearn.metrics
import numpy as np
import pandas as pd
from rdkit import Chem
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error

# Converting SMILES to Graph

In [ ]:
# Load dataset
data = pd.read_csv('/content/drive/MyDrive/ORGANS/BRAIN/Proccesed_Brain_Q00535.csv')

# Defining the smiles_to_graph function
def smiles_to_graph(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        return None
    molecule = Chem.AddHs(molecule)

    num_atoms = molecule.GetNumAtoms()

    # Simple feature representation: one-hot encoding for atom types (C, O, N, B)
    atom_features = np.zeros((num_atoms, 4))

    for atom in molecule.GetAtoms():
        atom_type = atom.GetSymbol()
        if atom_type == 'C':
            atom_features[atom.GetIdx()][0] = 1
        elif atom_type == 'O':
            atom_features[atom.GetIdx()][1] = 1
        elif atom_type == 'N':
            atom_features[atom.GetIdx()][2] = 1
        elif atom_type == 'B':
            atom_features[atom.GetIdx()][3] = 1

    bond_indices = []
    bond_features = []

    for bond in molecule.GetBonds():
        start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        bond_indices.extend([(start, end), (end, start)])  # Adding edges for both directions here

        # Bond feature representation
        bond_type = bond.GetBondType()
        is_in_ring = bond.IsInRing()

        bond_feature = [
            1 if bond_type == Chem.rdchem.BondType.SINGLE else 0,
            1 if bond_type == Chem.rdchem.BondType.DOUBLE else 0,
            1 if bond_type == Chem.rdchem.BondType.TRIPLE else 0,
            1 if bond_type == Chem.rdchem.BondType.AROMATIC else 0,
            1 if is_in_ring else 0
        ]

        bond_features.extend([bond_feature, bond_feature.copy()])  # Adding feature for both bond directions here

    return {
        'atom_features': atom_features,
        'bond_indices': bond_indices,
        'bond_features': bond_features,
    }


In [ ]:
# Converting SMILES strings to molecular graphs and creating a list of Data objects
data_list = []
for index, row in data.iterrows():
    smiles = row['SMILES']
    affinity = row['Y(Obs)']
    graph = smiles_to_graph(smiles)
    if graph is not None:
        x = torch.tensor(graph['atom_features'], dtype=torch.float32)
        edge_index = torch.tensor(graph['bond_indices'], dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(graph['bond_features'], dtype=torch.float32)
        y = torch.tensor([affinity], dtype=torch.float32)
        data_list.append(Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y))


# GCN Model

In [ ]:
# 2 layer GNN model
class GNNModel(nn.Module):
    def __init__(self, num_features, hidden_dim, dropout_rate=0.5):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = self.dropout(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.dropout(x)
        x = global_mean_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Cross Validation

In [ ]:
# Cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initializing the model and training parameters
batch_size = 32
num_epochs = 150
patience = 25

In [ ]:
# Cross-validation loop
for fold, (train_idx, val_idx) in enumerate(kf.split(data_list)):
    train_data = DataLoader([data_list[i] for i in train_idx], batch_size=batch_size, shuffle=True)
    val_data = DataLoader([data_list[i] for i in val_idx], batch_size=batch_size, shuffle=False)

    model = GNNModel(num_features=4, hidden_dim=128, dropout_rate=0.5)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = nn.MSELoss()

    best_val_loss = np.inf
    best_epoch = -1
    early_stop_counter = 0
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for data in train_data:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, data.y.view(-1, 1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_data)
        train_losses.append(avg_loss)

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for data in val_data:
                output = model(data)
                val_loss = criterion(output, data.y.view(-1, 1))
                total_val_loss += val_loss.item()
        avg_val_loss = total_val_loss / len(val_data)
        val_losses.append(avg_val_loss)

        # Checkpoint model if validation loss improved
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_epoch = epoch
            torch.save(model.state_dict(), f'best_gnn_model_fold_{fold}.pth')
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f'Early stopping at epoch {epoch + 1}')
                break


In [ ]:
# Loading the best model and evaluating on the validation set
model.load_state_dict(torch.load(f'best_gnn_model_fold_{fold}.pth'))
model.eval()
actual_affinities = []
predicted_affinities = []
with torch.no_grad():
  for data in val_data:

    output = model(data)
    actual_affinities.extend(data.y.view(-1).cpu().numpy())
    predicted_affinities.extend(output.view(-1).cpu().numpy())

In [ ]:
    rmse = np.sqrt(mean_squared_error(actual_affinities, predicted_affinities))
    mae = mean_absolute_error(actual_affinities, predicted_affinities)

    # Append per-fold results to check later
    fold_results['rmse_scores'].append(rmse)
    fold_results['mae_scores'].append(mae)
    fold_results['actual_affinities'].extend(actual_affinities)
    fold_results['predicted_affinities'].extend(predicted_affinities)

    # Ploting training/validation loss
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Fold {fold+1} Training and Validation Loss')
    plt.legend()
    plt.show()


In [ ]:
# Calculate and print overall evaluation metrics
overall_rmse = np.sqrt(mean_squared_error(fold_results['actual_affinities'], fold_results['predicted_affinities']))
overall_mae = mean_absolute_error(fold_results['actual_affinities'], fold_results['predicted_affinities'])
print(f'Overall RMSE: {overall_rmse:.4f}')
print(f'Overall MAE: {overall_mae:.4f}')